In [1]:
#LIBRARY IMPORT
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import re
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
import pip
pip.main(["install", "openpyxl"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable

Requirement already satisfied: openpyxl in /home/adi/.local/lib/python3.10/site-packages (3.1.3)

Requirement already satisfied: et-xmlfile in /home/adi/.local/lib/python3.10/site-packages (from openpyxl) (1.1.0)

0

In [3]:
#reading data from excel file
data = pd.read_excel('./kaggle/input/datasets/dataset.xlsx').sample(5000)

#DATA PREPROCESSING
#removing duplicates
data.drop_duplicates(inplace=True)

#remove all duplicates
print("Number of duplicate rows: ", data.duplicated().sum())

#Delete Duplicates
data = data.drop_duplicates(subset="track_id", keep='first', inplace=False)


Number of duplicate rows:  0


In [4]:
#handle missing values
data = data.dropna()

#REGULAR EXPRESSION
def clean_text(text):
    return re.sub(r'[\x00-\x1F\x7F-\xFF]', '', str(text).replace(';', ''))

data['artists'] = data['artists'].apply(lambda x: clean_text(x))
data['album_name'] = data['album_name'].apply(lambda x: clean_text(x))
data['track_name'] = data['track_name'].apply(lambda x: clean_text(x))


In [5]:
#FEATURE SELECTION
selected_columns = ['artists', 'track_name', 'popularity', 'danceability', 'energy', 'acousticness', 'tempo', 'valence', 'explicit', 'track_genre']
reduced_data = data[selected_columns]

# Text feature vectorization
text_features = ['artists','track_genre']
tfidf_vectorizer = TfidfVectorizer()
tfidf_text_features = tfidf_vectorizer.fit_transform(reduced_data[text_features].apply(' '.join, axis=1))

# Feature selection
selected_features = ['popularity', 'danceability', 'energy', 'acousticness', 'tempo', 'valence', 'explicit']
numeric_features = reduced_data[selected_features]


In [6]:
# Convert TF-IDF features to DataFrame
tfidf_df = pd.DataFrame(tfidf_text_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Combine text and numeric features
X = pd.concat([tfidf_df, numeric_features.reset_index(drop=True)], axis=1)
y = reduced_data['track_name'].reset_index(drop=True)

# Data saving and loading
with open('dataofmodel.txt', 'wb') as f1:
    pickle.dump((X, y), f1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:

# Model training
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)


: 

In [ ]:


# Model evaluation
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*14

print("Accuracy:", accuracy)

Accuracy: 0.4392712550607287


In [ ]:
joblib.dump(rf_classifier, 'edai_model_joblib')

user_artist = input("Enter the artist name: ")
user_genre = input("Enter the genre: ")

print("Predicted Track Names (First 5):")
predictions_df = pd.DataFrame({'Index': range(len(y_pred[:5])), 'Track Name': y_pred[:5]})
print(predictions_df)


predictions_df.to_csv("predictions.csv", index=False)

Enter the artist name:  Gen Hoshino
Enter the genre:  acoustic


Predicted Track Names (First 5):
   Index                    Track Name
0      0  Merry Christmas, Kiss My Ass
1      1                      Amarillo
2      2                 Soar Nada Mas
3      3                  Made of Lies
4      4          Why We Like Spelling


#tring to code if songs can be given on bais of input

In [ ]:
!ls


dataofmodel.txt  edai_model_joblib  predictions.csv


FLASK BACKEND FOR WEBSITE


In [ ]:
from flask import Flask, jsonify, request
import joblib

app = Flask(__name__)

# Load the model
model = joblib.load('edai_model_joblib')

@app.route('/recommend', methods=['POST'])
def get_recommendations():
    data = request.json
    if not data:
        return jsonify({'error': 'No input data provided'}), 400

    artist = data.get('artist')
    genre = data.get('genre')
    area = data.get('area')

    if not artist or not genre or not area:
        return jsonify({'error': 'Please provide artist, genre, and area'}), 400

    try:
        # Call the model's recommendation function with the provided parameters
        recommendations = model.get_recommendations(artist, genre, area)
        return jsonify(recommendations)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 654, in initialize
    self.init_sockets()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/opt/conda/lib/python3.10/site-packages/

SystemExit: 1

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import subprocess
import time
import requests

# Ensure Flask app code is defined in a cell above this
# Save Flask app code in 'app.py' for subprocess to run it

# Write the Flask app code to a Python file
with open('app1.py', 'w') as f:
    f.write("""
from flask import Flask, jsonify, request
import joblib

app = Flask(__name__)

# Load the model
model = joblib.load('edai_model_joblib')

@app.route('/recommend', methods=['POST'])
def get_recommendations():
    data = request.json
    if not data:
        return jsonify({'error': 'No input data provided'}), 400

    artist = data.get('artist')
    genre = data.get('genre')
    area = data.get('area')

    if not artist or not genre or not area:
        return jsonify({'error': 'Please provide artist, genre, and area'}), 400

    try:
        # Call the model's recommendation function with the provided parameters
        recommendations = model.get_recommendations(artist, genre, area)
        return jsonify(recommendations)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)
    """)

# Run the Flask app in the background
proc = subprocess.Popen(["python", "app1.py"])
time.sleep(5)  # Give the server a few seconds to start

# Example request to the Flask app
url = 'file:///C:/Users/bsrao/OneDrive/Desktop/mockdjangomusicwebapp/index2.html'
data = {
    'artist': 'Artist Name',
    'genre': 'Genre',
    'area': 'Area'
}
response = requests.post(url, json=data)
print(response.json())

# After testing, kill the Flask server
proc.kill()


InvalidSchema: No connection adapters were found for 'file:///C:/Users/bsrao/OneDrive/Desktop/mockdjangomusicwebapp/index2.html'